In [4]:
import kagglehub
print(kagglehub.__version__)

0.3.13


In [5]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "Mall_Customers.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "vjchoudhary7/customer-segmentation-tutorial-in-python",
  file_path,
)

df.head()


C:\Users\yoges\AppData\Local\Temp\ipykernel_13396\433474431.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [6]:
df.shape

(200, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [8]:
df.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
# as it show there is No duplicate

In [11]:
df.columns = df.columns.str.lower()

In [12]:
df["gender"] = df["gender"].str.strip()

In [13]:
df.rename(columns = {"customerid":"customer_id",
                     "annual income (k$)":"annual_income($k)",
                     "spending score (1-100)":"score"},inplace = True)

In [14]:
df["gender"].value_counts()

gender
Female    112
Male       88
Name: count, dtype: int64

In [15]:
df["annual_income"] =  df["annual_income($k)"]*1000

In [16]:
df.head(10)

,customer_id,gender,age,annual_income($k),score,annual_income
0,1,Male,19,15,39,15000
1,2,Male,21,15,81,15000
2,3,Female,20,16,6,16000
3,4,Female,23,16,77,16000
4,5,Female,31,17,40,17000
5,6,Female,22,17,76,17000
6,7,Female,35,18,6,18000
7,8,Female,23,18,94,18000
8,9,Male,64,19,3,19000
9,10,Female,30,19,72,19000


In [17]:
def age_group(age):
    if age <= 30:
        return "18-30"
    elif age <= 50:
        return "31-50"
    elif age <= 70:
        return "51-70"
    else:
        return "+70 "
    
df["age_group"] = df["age"].apply(age_group)

In [34]:
df.to_csv("cleaned_mall_customers.csv", index=False)

In [18]:
print(df["age"].agg(["min","max"]))

min    18
max    70
Name: age, dtype: int64


In [19]:
df.groupby("gender")["annual_income($k)"].agg(gender_count = "count",average_salary = "mean",total_income_k = "sum")

,gender_count,average_salary,total_income_k
gender,,,
Female,112,59.250000,6636
Male,88,62.227273,5476


In [20]:
df.groupby("gender").agg({"annual_income($k)": ["count","mean","sum"],
                          "score": ["mean","max"]
                          })

annual_income($k)                       score    
                   count       mean   sum       mean max
gender                                                  
Female               112  59.250000  6636  51.526786  99
Male                  88  62.227273  5476  48.511364  97

In [21]:
df.pivot_table(values="annual_income($k)",index="age_group",columns="gender",aggfunc= "count")

gender,Female,Male
age_group,,
18-30,34,28
31-50,60,38
51-70,18,22


In [22]:
df.pivot_table(values="annual_income($k)",index="age_group",columns="gender",aggfunc= sum)

C:\Users\yoges\AppData\Local\Temp\ipykernel_13396\995768438.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df.pivot_table(values="annual_income($k)",index="age_group",columns="gender",aggfunc= sum)


gender,Female,Male
age_group,,
18-30,1766,1657
31-50,3844,2691
51-70,1026,1128


In [23]:
df.pivot_table(values="annual_income($k)",index="age_group",columns="gender",aggfunc= ["min","max"])

min         max     
gender    Female Male Female Male
age_group                        
18-30         16   15     98  137
31-50         17   20    126  137
51-70         20   19    101   93

In [24]:
df.pivot_table(values="annual_income($k)",index="age_group",columns="gender",aggfunc= ["mean","median"])

mean            median      
gender        Female       Male Female  Male
age_group                                   
18-30      51.941176  59.178571   55.5  60.5
31-50      64.066667  70.815789   63.0  72.0
51-70      57.000000  51.272727   53.5  51.5

In [25]:
df.pivot_table(values="score",index="age_group",columns="gender",aggfunc= ["mean","median","min","max"])

mean            median          min         max     
gender        Female       Male Female  Male Female Male Female Male
age_group                                                           
18-30      62.735294  58.964286   68.5  59.0      6    5     94   97
31-50      48.900000  47.973684   45.5  47.0      5    1     99   95
51-70      39.111111  36.136364   45.5  46.0      5    3     59   60

In [26]:
df.pivot_table(values=["annual_income($k)","score"],index="age_group",columns="gender",aggfunc= "max")

annual_income($k)       score     
gender               Female Male Female Male
age_group                                   
18-30                    98  137     94   97
31-50                   126  137     99   95
51-70                   101   93     59   60

In [27]:
df.pivot_table(values=["annual_income($k)","score"],index="age_group",columns="gender",aggfunc={"annual_income($k)":"sum","score":"max"})

annual_income($k)        score     
gender               Female  Male Female Male
age_group                                    
18-30                  1766  1657     94   97
31-50                  3844  2691     99   95
51-70                  1026  1128     59   60

In [28]:
idx = df.groupby(['age_group','gender'])['score'].idxmax()

In [29]:
idx

age_group  gender
18-30      Female      7
           Male      145
31-50      Female     11
           Male      127
51-70      Female     63
           Male       53
Name: score, dtype: int64

In [30]:
idx2 = df.groupby(['age_group','gender'])['score'].idxmin()

In [31]:
max_pivot = df.loc[idx,["age_group","gender","annual_income($k)","score"]]
max_pivot

,age_group,gender,annual_income($k),score
7,18-30,Female,18,94
145,18-30,Male,77,97
11,31-50,Female,19,99
127,31-50,Male,71,95
63,51-70,Female,47,59
53,51-70,Male,43,60


In [32]:
min_pivot = df.loc[idx2,["age_group","gender","annual_income($k)","score"]]
min_pivot

,age_group,gender,annual_income($k),score
2,18-30,Female,16,6
134,18-30,Male,73,5
22,31-50,Female,25,5
156,31-50,Male,78,1
140,51-70,Female,75,5
8,51-70,Male,19,3


##EDA

In [33]:
df1 = df.groupby("gender")[["annual_income($k)","score"]].mean()
df1.rename(columns={"annual_income($k)":"Average_income","score":"Average_score"},inplace=True)
df1

,Average_income,Average_score
gender,,
Female,59.250000,51.526786
Male,62.227273,48.511364
